### Incremental Project_3 (2014136131_추명엽_파이썬01분반)

In [2]:
file = open('Eng_StopWords.txt','r')
s = file.read()
Eng_StopWords = s.split()
file.close()

In [3]:
file = open('Kor_StopWords.txt','r')
s = file.read()
Kor_StopWords = s.split()
file.close()

In [4]:
import requests
import string

req = requests.get('https://cse.koreatech.ac.kr/')
source1 = req.text #컴공 홈페이지
req = requests.get('https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311')
source2 = req.text#초현실주의 사진 14만명을 사로잡음
req = requests.get('http://www.etnews.com/20181115000026?mc=ns_005_00004')
source3 = req.text#반자율주행 실현한 볼보자동차 '지능형 안전기술'
req = requests.get('http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/')
source4 = req.text#말 안통하는 사람의 특징 7
req = requests.get('https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html')
source5 = req.text#Makr Zuckerberg Defends Facebook

In [5]:
file = open('cse.koreatech.ac.html.txt','w',encoding='UTF-8')
file.write(source1)
file.close()

In [6]:
file = open('post.naver.com.html.txt','w',encoding='UTF-8')
file.write(source2)
file.close()

In [7]:
file = open('www.etnews.com.html.txt','w')
file.write(source3)
file.close()

In [8]:
file = open('kormedi.com.txt','w',encoding='UTF-8')
file.write(source4)
file.close()

In [9]:
file = open('www.nytimes.com.html.txt','w',encoding='UTF-8')
file.write(source5)
file.close()

In [10]:
def pureWord(source):
    l = "" #태그가 지워진 HTML코드가 저장될 문자열
    s = [] #<의 개수를 저장할 리스트
    count = 0 #문자를 하나씩 읽을때, 현재 문자의 위치를 저장할 변수
    isStyle = False # <style> 태그가 시작되었는지 확인
    isScript = False # <script>태그가 시작되었는지 확인
    punctuation = string.punctuation # 구두문자들
    
    #Stack의 개념을 통하여 문제 해결(단, HTML태그 안에서 < >를 단어취급 하고 싶으면 이스케이프와 같이 예외처리해야함)
    for y in source:
        if (y == '<' ): # 꺽쇠괄호 안에 있는 문자들은 모두 HTML 태그로 인식함
            if(not(isScript) and source[count+1]=='s' and source[count+2]=='c' and source[count+3]=='r' and source[count+4]=='i' and source[count+5]=='p' and source[count+6]=='t'):
                count = count + 1
                isScript = True # <script>태그가 나오면 isScript를 True로
                s.append(y) # <script> 태서에서 앞에 나온 <를 지우기 위해서
                continue
            if(not(isStyle) and source[count+1]=='s'and source[count+2]=='t' and source[count+3]=='y' and source[count+4]=='l' and source[count+5]=='e'):
                count = count +1
                isStyle = True # <style>태그가 나오면 isStyle를 True로
                s.append(y) # <style>태그에서 앞에 나온 <를 지우기 위해서
                continue
            if(source[count-4] =='<' and source[count-3] =='!' and source[count-2] =='-' and source[count-1] =='-' and source[count+1]=='!'):
                count = count + 1 #HTML의 태그중<!--<!이 나오는 경우 >과로를 하나만 쓰는 경우가 있어 예외처리함
                continue
            if (not(isScript) and not(isStyle)): # <script>태그와 <style>태그가 시작하면 태그들이 끝날때 까지 내용 무시
                count = count + 1
                s.append(y) #< > 꺽쇠괄호는 태그로 인식하고 지운다.
                continue
        if (s == [] and not(isScript) and not(isStyle) and not(y in punctuation)):
            l = l + y # < > 꺽쇠괄호 안에 있지 않고 <script>와 <html>태그 안에 있지 않으며 구두문자가 아니면 의미있는 문자
        if y == '>' :
            if (count > 0 and source[count-1]=='='): # => 가 나오면 무시
                count = count + 1
                continue
            if ((count-6) > 0 and (isScript or isStyle)): # count가 6 이상이면 </stlye>과 </script>가 나올 수 있다.
                if (source[count-1]=='t' and source[count-2]=='p' and source[count-3]=='i' and source[count-4]=='r' and source[count-5]=='c'and source[count-6]=='s' and source[count-7]=='/'):
                    isScript = False
                    s.pop()
                    count = count + 1
                    continue
                if (source[count-1]=='e' and source[count-2]=='l'and source[count-3]=='y' and source[count-4]=='t' and source[count-5]=='s' and source[count-6]=='/'):
                    isStyle = False
                    count = count + 1
                    s.pop()
                    continue
            if (not(isScript) and not(isStyle)): #<script>태그와 <style>태그 안에서는 내용 무시
                count = count + 1
                s.pop() # 꺽쇠괄호가 끝났는지 확인
                continue
        count = count + 1 # 단어 하나를 읽음
    pureWords = l.split() # 공백문자를 기준으로 단어를 나눔
    countWord = {}
    for x in pureWords:
        if(x in countWord): #사전에 있는 단어이면 value를 증가
            countWord[x] = countWord[x] + 1
        else: #사전에 없는 단어이면 단어를 키워드로 value를 1로 저장
            countWord[x] = 1
    return countWord

In [11]:
#stopword가 list 형태의 pureWords 안에 있으면 해당 단어의 빈도수를 0으로 만듬
def delete_stopWords(pureWords):
    for key in pureWords:
        if(key in Eng_StopWords):
            pureWords[key]=0
        if(key in Kor_StopWords):
            pureWords[key]=0

In [17]:
pureWords_1 = pureWord(source1)

In [18]:
pureWords_2 = pureWord(source2)

In [19]:
pureWords_3 = pureWord(source3)

In [20]:
pureWords_4 = pureWord(source4)

In [21]:
pureWords_5 = pureWord(source5)

In [22]:
import pickle
# stopword의 빈도수가 0이된 pureword 리스트를 객체 형태로 텍스트 파일에 저장
f = open('cse.koreatech.ac.words_frequency.pickle','wb')
pickle.dump(pureWords_1, f)
f.close()

f = open('cse.koreatech.ac.words_frequency.pickle','rb')
pickle_words1 = pickle.load(f)
f.close()

In [23]:
f = open('post.naver.com.words_frequency.pickle','wb')
pickle.dump(pureWords_2, f)
f.close()

f = open('post.naver.com.words_frequency.pickle','rb')
pickle_words2 = pickle.load(f)
f.close()

In [24]:
f = open('www.etnews.com.words_frequency.pickle','wb')
pickle.dump(pureWords_3, f)
f.close()

f = open('www.etnews.com.words_frequency.pickle','rb')
pickle_words3 = pickle.load(f)
f.close()

In [25]:
f = open('kormedi.com.words_frequency.pickle','wb')
pickle.dump(pureWords_4, f)
f.close()

f = open('kormedi.com.words_frequency.pickle','rb')
pickle_words4 = pickle.load(f)
f.close()

In [26]:
f = open('www.nytimes.com.words_frequency.pickle','wb')
pickle.dump(pureWords_5, f)
f.close()

f = open('www.nytimes.com.words_frequency.pickle','rb')
pickle_words5 = pickle.load(f)
f.close()

In [27]:
def export_decendOrder(pureWords): #사전에 있는 단어의 출현 빈도수를 기준으로 내림차순 정리
    max_val = [0, 0, 0] # Top3 출현 빈도수
    max_key = ['NULL', 'NULL', 'NULL'] # Top3 출현 단어
    temp_val = 0 # Top3 출현 빈도수를 바꾸기 위한 변수
    temp_key = 'key' #Top3 출현 단어를 바꾸기 위한 변수
    for key in pureWords:
        length = len(max_val) #Top3의 index를 뒤에서 부터 접근하기 위해
        count = 0
        if(pureWords[key] > max_val[0]): #Top1의 값이 바뀐 경우
            for val in max_val: #Top3가 바뀌면 바뀐 index 뒤에있는 값들이 한칸씩 뒤로간다
                count = count + 1
                max_val[length-count] = max_val[length-count-1]
                max_key[length-count] = max_key[length-count-1]
                if(count > 1): # 값을 하나씩 미루는 것을 2번 하면 종료
                    count = 0
                    break
            max_val[0] = pureWords[key] #첫번째 값을 바꿈
            max_key[0] = key
            continue #Top3에 대해 변경하였으면 다음 단어의 Top3 포함 유무를 판단
        if(pureWords[key] > max_val[1]): #Top2의 값이 바뀐 경우
            for val in max_val:
                count = count + 1
                max_val[length-count] = max_val[length-count-1]
                max_key[length-count] = max_key[length-count-1]
                if(count > 0): #값을 하나씩 미루는 것을 1번 하면 종료
                    count = 0
                    break
            max_val[1] = pureWords[key] #두번째 값을 바꿈
            max_key[1] = key
            continue #Top3에 대해 변경하였으면 다음 단어의 Top3 포함 유무를 판단
        if(pureWords[key] > max_val[2]): #Top3의 값이 바뀐 경우
            max_val[2] = pureWords[key] # 3번째 값을 바꿈
            max_key[2] = key
            continue #Top3에 대해 변경하였으면 다음 단어의 Top3 포함 유무를 판단
    return max_key


In [28]:
# Top3 출현 빈도 단어
delete_stopWords(pickle_words1)
top3_words1 = export_decendOrder(pickle_words1)
print(top3_words1)

['컴퓨터공학부', '20181125', '1700']


In [29]:
delete_stopWords(pickle_words2)
top3_words2 = export_decendOrder(pickle_words2)
print(top3_words2)

['네이버', '포스트', '초현실주의']


In [30]:
delete_stopWords(pickle_words3)
top3_words3 = export_decendOrder(pickle_words3)
print(top3_words3)

['전자신문', '차량이', '도로']


In [33]:
delete_stopWords(pickle_words4)
top3_words4 = export_decendOrder(pickle_words4)
print(top3_words4)

['건강뉴스', '사람의', '된다']


In [32]:
delete_stopWords(pickle_words5)
top3_words5 = export_decendOrder(pickle_words5)
print(top3_words5)

['Zuckerberg', 'Mr', 'Facebook']


In [35]:
category_sience_words = ['자율주행','과학','전자','시스템'] # sience 카테고리 판단 단어들
category_car_words = ['자동차','도로','차량','자율주행','주행','운전','탑승','브레이크','계기판','차선'] # car 카테고리 판단 단어들
category_education_words = ['학교','edu','학부','학생','학사','학원','공학','교육'] # education 카테고리 판단 단어들
category_IT_words = ['디지털','통신','It','IT','컴퓨터','소프트웨어','인공지능','Facebook','community','IoT','Twitter']
# IT 카테고리 판단 단어들
category_art_words = ['초현실','사진','아티스트','예쁜']# art 카테고리 판단 단어들
category_health_words = ['자신감','근력','건강','말','바람직','행동','정직','헬스','무례','완치','장애','치매','사람의']# health 카테고리 판단 단어들
category_issue_words = ['competition','problem','laws','Federal','committee','policy','People','Twitter','오피니언','문의','자살','유가족','치매']
# issue 카테고리 판단 단어들
def decide_category(dict_words): # 사전에 들어있는 단어들이 어떤 카테고리에 가까운지 판단
    category_similarity_list = {} # 각각의 카테고리와의 유사도 모음
    #각각의 카테고리와의 유사도 모음
    sience_count = 0
    car_count = 0
    edu_count = 0
    IT_count = 0
    art_count = 0
    health_count = 0
    issue_count = 0
    for dict_word in dict_words:
        #카테고리의 단어들이 사전에 있는 단어들에 들어 있으면 해당 카테고리의 빈도수를 증가
        for category_word in category_sience_words:
            if (category_word in dict_word):
                sience_count = sience_count + 1
        for category_word in category_car_words:
            if (category_word in dict_word):
                car_count = car_count + 1
        for category_word in category_education_words:
            if (category_word in dict_word):
                edu_count = edu_count + 1
        for category_word in category_IT_words:
            if (category_word in dict_word):
                IT_count = IT_count + 1
        for category_word in category_art_words:
            if (category_word in dict_word):
                art_count = art_count + 1
        for category_word in category_health_words:
            if (category_word in dict_word):
                health_count = health_count + 1
        for category_word in category_issue_words:
            if (category_word in dict_word):
                issue_count = issue_count + 1
    # 각각의 카테고리와의 유사도 모음 등록
    category_similarity_list['sience']= sience_count
    category_similarity_list['edu']= edu_count
    category_similarity_list['car']= car_count
    category_similarity_list['IT']= IT_count
    category_similarity_list['art']= art_count
    category_similarity_list['health']= health_count
    category_similarity_list['issue']= issue_count
    return  category_similarity_list  #해당 사전을 각각의 카테고리와의 유사도 모음 반환

In [36]:
# 5개의 사전들의 Top3 카테고리를 유사도의 내림차순으로 추출
category_words1_top3 = export_decendOrder(decide_category(pickle_words1))
category_words2_top3 = export_decendOrder(decide_category(pickle_words2))
category_words3_top3 = export_decendOrder(decide_category(pickle_words3))
category_words4_top3 = export_decendOrder(decide_category(pickle_words4))
category_words5_top3 = export_decendOrder(decide_category(pickle_words5))
print(category_words1_top3)
print(category_words2_top3)
print(category_words3_top3)
print(category_words4_top3)
print(category_words5_top3)

['edu', 'IT', 'sience']
['art', 'NULL', 'NULL']
['car', 'sience', 'IT']
['health', 'issue', 'edu']
['IT', 'issue', 'edu']


In [39]:
def cmp_by_each_element(a): #유사도가 사전의 values에 저장이 된다
    return a[1]
def search(): #검색 문장의 연관 link를 유사도의 내림차순으로 출력
    statement = input()
    urls = ['https://cse.koreatech.ac.kr/','https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311','http://www.etnews.com/20181115000026?mc=ns_005_00004','http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/','https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html']
    #url에 대한 설명
    describe_url = {'https://cse.koreatech.ac.kr/':'#한기대 컴퓨터공학부',\
                   'https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311':'#초현실주의 사진 14만명을 사로잡음',\
                   'http://www.etnews.com/20181115000026?mc=ns_005_00004':"#반자율주행 실현한 볼보자동차 '지능형 안전기술'",\
                   'http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/':'#말 안통하는 사람의 특징 7',\
                   'https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html':"#Makr Zuckerberg Defends Facebook"}
    
    pure_words = pureWord(statement) # 검색 문장을 단어로 나눔
    delete_stopWords(pure_words) # 검색 문장 중에서 stopword를 제거
    top3_category_statement_words = export_decendOrder(decide_category(pure_words)) # 검색 문장의 Top3 카테고리
    top3_statement_words = export_decendOrder(pure_words) # 검색 문장의 Top3 단어
    
    BeSearchedObject_WordList = [top3_words1,top3_words2,top3_words3,top3_words4,top3_words5] # 각각의 링크의 Top3 단어
    BeSearchedObject_CategoryList = [category_words1_top3, category_words2_top3, category_words3_top3, category_words4_top3, category_words5_top3 ]
    # 각각의 링크의 Top3 카테고리
    
    score_word = {} # 검색 문장의 Top3 단어와 각각의 링크의 Top3 단어 유사도 모음 (유사도는 1.0 만점)
    score_category = {} # 검색 문장의 Top3 카테고리와 각각의 링크의 Top3 카테고리 유사도 모음 (유사도는 1.0 만점)
    score_similarity_total = {} # 단어의 유사도와 카테고리의 유사도를 합한 유사도 모음 (유사도는 2.0 만점)
    for index, x in enumerate(urls):
        score_word[urls[index]] = 0 #각각의 링크의 단어 유사도 0으로 초기화
    for index, x in enumerate(urls): 
        score_category[urls[index]] = 0 #각각의 링크의 카테고리 유사도 0으로 초기화
    for index, x in enumerate(urls):
        score_similarity_total[urls[index]] = 0 #각각의 링크의 총 유사도 0으로 초기화
    
    #각각의 링크의 단어 유사도 조사
    for index1, BeSearchedObject_Words in enumerate(BeSearchedObject_WordList): #각각의 링크별 Top3 단어사전
        for index2, BeSearchedObject_word in enumerate(BeSearchedObject_Words): #링크별 Top3 단어
            for index3, top3_statement_word in enumerate(top3_statement_words): #검색 문장의 Top3 단어를 링크의 Top3 단어들과 비교
                if (top3_statement_word in BeSearchedObject_word): # 검색 문장의 Top3 단어가 단어사전의 Top3 단어들에 언급 되었는지 판단
                    score_word[urls[index1]] = score_word[urls[index1]] + (18 - 3*(index2 + index3)) #index2와 index3가 top1이면 18점
        score_similarity_total[urls[index1]] = score_word[urls[index1]]/(18*(len(top3_statement_words)**2))#만점:18*(len(top3_statement_words)**2)
     
    #각각의 링크의 카테고리 유사도 조사
    for index1, BeSearchedObject_CategoryWords in enumerate(BeSearchedObject_CategoryList): #각각의 링크별 Top3 카테고리 모음
        for index2, BeSearchedObject_CategoryWord in enumerate(BeSearchedObject_CategoryWords):#링크별 Top3 카테고리
            for index3, top3_category_statement_word in enumerate(top3_category_statement_words): #검색 문장의 Top3 카테고리를 링크의 Top3 단어들과 비교
                if (top3_category_statement_word == 'NULL'):#검색문장의 top3 카테고리가 NULL인 경우 다음 카테고리 단어로 넘어간다
                    continue
                if (top3_category_statement_word in BeSearchedObject_CategoryWord):#검색 문장의 Top3 카테고리가 링크 Top3 카테고리 단어들에 언급 되었는지 판단
                    score_category[urls[index1]] = score_category[urls[index1]] + (18 - 3*(index2 + index3)) #index2와 index3가 top1이면 18점
        score_similarity_total[urls[index1]] += score_category[urls[index1]]/(18*(len(top3_statement_words)))#만점:18*(len(top3_statement_words))
    
    result = list(score_similarity_total.items())
    result.sort(key = cmp_by_each_element) #링크별 총 유사도를 기준으로 내림차순
    result.reverse() #총 유사도를 기준으로 내림차순 정렬되어있는 것을 순서를 반대로
    for url,freq in result: # 검색 문장과 유사도가 높은 url부터 출력
        print(url)
        print(describe_url[url], freq) # 해당 url의 설명과 총 유사도
        print()
    
    

In [40]:
search()

컴퓨터
https://cse.koreatech.ac.kr/
#한기대 컴퓨터공학부 0.3888888888888889

https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html
#Makr Zuckerberg Defends Facebook 0.3333333333333333

http://www.etnews.com/20181115000026?mc=ns_005_00004
#반자율주행 실현한 볼보자동차 '지능형 안전기술' 0.2222222222222222

http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/
#말 안통하는 사람의 특징 7 0.0

https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311
#초현실주의 사진 14만명을 사로잡음 0.0



In [41]:
search()

Facebook
https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html
#Makr Zuckerberg Defends Facebook 0.4074074074074074

https://cse.koreatech.ac.kr/
#한기대 컴퓨터공학부 0.2777777777777778

http://www.etnews.com/20181115000026?mc=ns_005_00004
#반자율주행 실현한 볼보자동차 '지능형 안전기술' 0.2222222222222222

http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/
#말 안통하는 사람의 특징 7 0.0

https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311
#초현실주의 사진 14만명을 사로잡음 0.0



In [42]:
search()

말을 잘하고 싶어요
http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/
#말 안통하는 사람의 특징 7 0.3333333333333333

https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html
#Makr Zuckerberg Defends Facebook 0.0

http://www.etnews.com/20181115000026?mc=ns_005_00004
#반자율주행 실현한 볼보자동차 '지능형 안전기술' 0.0

https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311
#초현실주의 사진 14만명을 사로잡음 0.0

https://cse.koreatech.ac.kr/
#한기대 컴퓨터공학부 0.0



In [43]:
search()

자율주행자동차의 비전을 알고싶어요
http://www.etnews.com/20181115000026?mc=ns_005_00004
#반자율주행 실현한 볼보자동차 '지능형 안전기술' 0.5555555555555556

https://cse.koreatech.ac.kr/
#한기대 컴퓨터공학부 0.16666666666666666

https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html
#Makr Zuckerberg Defends Facebook 0.0

http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/
#말 안통하는 사람의 특징 7 0.0

https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311
#초현실주의 사진 14만명을 사로잡음 0.0



In [44]:
search()

사진으로 사람을 주목시키는 방법
https://post.naver.com/viewer/postView.nhn?volumeNo=17087591&memberNo=4444311
#초현실주의 사진 14만명을 사로잡음 0.3333333333333333

https://www.nytimes.com/2018/11/15/technology/zuckerberg-facebook-sandberg-tactics.html
#Makr Zuckerberg Defends Facebook 0.0

http://kormedi.com/1255552/%EB%A7%90-%EC%95%88-%ED%86%B5%ED%95%98%EB%8A%94-%EC%82%AC%EB%9E%8C%EC%9D%98-%ED%8A%B9%EC%A7%95-7/
#말 안통하는 사람의 특징 7 0.0

http://www.etnews.com/20181115000026?mc=ns_005_00004
#반자율주행 실현한 볼보자동차 '지능형 안전기술' 0.0

https://cse.koreatech.ac.kr/
#한기대 컴퓨터공학부 0.0

